<a href="https://colab.research.google.com/github/hurricane195/Intro-to-Deep-Learning/blob/Homework_5/HW5_P3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this homework, we focus on sequence-to-sequence modeling, similar to homework 4.

Developed a Transformer-based encoder-decoder architecture for English to French Translation. Train the model on the entire dataset and evaluate it on the entire dataset. Report training loss, validation loss, and validation accuracy. Also, try some qualitative validation as well, asking the network to generate French translations for some English sentences. Compare your results against and RNN-based network with attention and without attention.

In [ ]:
#Using a modided example of Dr. Tabkhi's "RNN" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/RNN.py
#Using a modided example of Dr. Tabkhi's "RNN-CharDataset" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/RNN-CharDataset.py
#Using a modided example of Dr. Tabkhi's "shakespeare-loader.py" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/shakespeare-loader.py
#Using a modided example of Dr. Tabkhi's "transformer_encoder_nextcharactor" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/transformer_encoder_nextcharactor.py
#Using a modided example of Dr. Tabkhi's "TransnNextCharPrediction Positionalencoding" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/TransnNextCharPrediction%20Positionalencoding.py
#Using a modided example of Dr. Tabkhi's "TransRegressionPositionalEncoding" available at https://github.com/HamedTabkhi/Intro-to-DL/blob/main/TransRegressionPositionalEncoding.py
#Random help from Chat GPT on formatting, sytntax, etc.
#Random help from Chat Colab AI on formatting, sytntax, etc.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Vocabulary:
    def __init__(self):
        self.word2index = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2}
        self.index2word = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>"}
        self.word_count = {}
        self.n_words = 30

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.word_count[word] = 1
            self.n_words += 1
        else:
            self.word_count[word] += 1

def tokenize_and_pad(sentences, vocab):
    max_length = max(len(sentence.split(' ')) for sentence in sentences) + 2
    tokenized_sentences = []
    for sentence in sentences:
        tokens = [vocab.word2index["<SOS>"]] + [vocab.word2index.get(word, vocab.word2index["<PAD>"]) for word in sentence.split(' ')] + [vocab.word2index["<EOS>"]]
        padded_tokens = tokens + [vocab.word2index["<PAD>"]] * (max_length - len(tokens))
        tokenized_sentences.append(padded_tokens)
    return torch.tensor(tokenized_sentences, dtype=torch.long)

In [ ]:
class EngFrDataset(Dataset):
    def __init__(self, pairs):
        self.eng_vocab = Vocabulary()
        self.fr_vocab = Vocabulary()
        self.pairs = []

        for eng, fr in pairs:
            self.eng_vocab.add_sentence(eng)
            self.fr_vocab.add_sentence(fr)
            self.pairs.append((eng, fr))

        self.eng_sentences = [pair[0] for pair in self.pairs]
        self.fr_sentences = [pair[1] for pair in self.pairs]

        self.eng_tokens = tokenize_and_pad(self.eng_sentences, self.eng_vocab)
        self.fr_tokens = tokenize_and_pad(self.fr_sentences, self.fr_vocab)

        self.eng_embedding = torch.nn.Embedding(self.eng_vocab.n_words, 100)
        self.fr_embedding = torch.nn.Embedding(self.fr_vocab.n_words, 100)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        eng_tokens = self.eng_tokens[idx]
        fr_tokens = self.fr_tokens[idx]
        eng_emb = self.eng_embedding(eng_tokens)
        fr_emb = self.fr_embedding(fr_tokens)
        return eng_tokens, fr_tokens, eng_emb, fr_emb

In [ ]:
english_to_french = [
     ("I am cold", "J'ai froid"),
    ("You are tired", "Tu es fatigué"),
    ("He is hungry", "Il a faim"),
    ("She is happy", "Elle est heureuse"),
    ("We are friends", "Nous sommes amis"),
    ("They are students", "Ils sont étudiants"),
    ("The cat is sleeping", "Le chat dort"),
    ("The sun is shining", "Le soleil brille"),
    ("We love music", "Nous aimons la musique"),
    ("She speaks French fluently", "Elle parle français couramment"),
    ("He enjoys reading books", "Il aime lire des livres"),
    ("They play soccer every weekend", "Ils jouent au football chaque week-end"),
    ("The movie starts at 7 PM", "Le film commence à 19 heures"),
    ("She wears a red dress", "Elle porte une robe rouge"),
    ("We cook dinner together", "Nous cuisinons le dîner ensemble"),
    ("He drives a blue car", "Il conduit une voiture bleue"),
    ("They visit museums often", "Ils visitent souvent des musées"),
    ("The restaurant serves delicious food", "Le restaurant sert une délicieuse cuisine"),
    ("She studies mathematics at university", "Elle étudie les mathématiques à l'université"),
    ("We watch movies on Fridays", "Nous regardons des films le vendredi"),
    ("He listens to music while jogging", "Il écoute de la musique en faisant du jogging"),
    ("They travel around the world", "Ils voyagent autour du monde"),
    ("The book is on the table", "Le livre est sur la table"),
    ("She dances gracefully", "Elle danse avec grâce"),
    ("We celebrate birthdays with cake", "Nous célébrons les anniversaires avec un gâteau"),
    ("He works hard every day", "Il travaille dur tous les jours"),
    ("They speak different languages", "Ils parlent différentes langues"),
    ("The flowers bloom in spring", "Les fleurs fleurissent au printemps"),
    ("She writes poetry in her free time", "Elle écrit de la poésie pendant son temps libre"),
    ("We learn something new every day", "Nous apprenons quelque chose de nouveau chaque jour"),
    ("The dog barks loudly", "Le chien aboie bruyamment"),
    ("He sings beautifully", "Il chante magnifiquement"),
    ("They swim in the pool", "Ils nagent dans la piscine"),
    ("The birds chirp in the morning", "Les oiseaux gazouillent le matin"),
    ("She teaches English at school", "Elle enseigne l'anglais à l'école"),
    ("We eat breakfast together", "Nous prenons le petit déjeuner ensemble"),
    ("He paints landscapes", "Il peint des paysages"),
    ("They laugh at the joke", "Ils rient de la blague"),
    ("The clock ticks loudly", "L'horloge tic-tac bruyamment"),
    ("She runs in the park", "Elle court dans le parc"),
    ("We travel by train", "Nous voyageons en train"),
    ("He writes a letter", "Il écrit une lettre"),
    ("They read books at the library", "Ils lisent des livres à la bibliothèque"),
    ("The baby cries", "Le bébé pleure"),
    ("She studies hard for exams", "Elle étudie dur pour les examens"),
    ("We plant flowers in the garden", "Nous plantons des fleurs dans le jardin"),
    ("He fixes the car", "Il répare la voiture"),
    ("They drink coffee in the morning", "Ils boivent du café le matin"),
    ("The sun sets in the evening", "Le soleil se couche le soir"),
    ("She dances at the party", "Elle danse à la fête"),
    ("We play music at the concert", "Nous jouons de la musique au concert"),
    ("He cooks dinner for his family", "Il cuisine le dîner pour sa famille"),
    ("They study French grammar", "Ils étudient la grammaire française"),
    ("The rain falls gently", "La pluie tombe doucement"),
    ("She sings a song", "Elle chante une chanson"),
    ("We watch a movie together", "Nous regardons un film ensemble"),
    ("He sleeps deeply", "Il dort profondément"),
    ("They travel to Paris", "Ils voyagent à Paris"),
    ("The children play in the park", "Les enfants jouent dans le parc"),
    ("She walks along the beach", "Elle se promène le long de la plage"),
    ("We talk on the phone", "Nous parlons au téléphone"),
    ("He waits for the bus", "Il attend le bus"),
    ("They visit the Eiffel Tower", "Ils visitent la tour Eiffel"),
    ("The stars twinkle at night", "Les étoiles scintillent la nuit"),
    ("She dreams of flying", "Elle rêve de voler"),
    ("We work in the office", "Nous travaillons au bureau"),
    ("He studies history", "Il étudie l'histoire"),
    ("They listen to the radio", "Ils écoutent la radio"),
    ("The wind blows gently", "Le vent souffle doucement"),
    ("She swims in the ocean", "Elle nage dans l'océan"),
    ("We dance at the wedding", "Nous dansons au mariage"),
    ("He climbs the mountain", "Il gravit la montagne"),
    ("They hike in the forest", "Ils font de la randonnée dans la forêt"),
    ("The cat meows loudly", "Le chat miaule bruyamment"),
    ("She paints a picture", "Elle peint un tableau"),
    ("We build a sandcastle", "Nous construisons un château de sable"),
    ("He sings in the choir", "Il chante dans le chœur")
]

In [ ]:
dataset = EngFrDataset(english_to_french)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, nhead, num_layers):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output[:, -1, :])  # Get the output of the last Transformer block
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

In [ ]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output[:, -1, :])  # Get the output of the last Transformer block
        return output, hidden

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

In [ ]:
# Hyperparameters
num_layers = 3
nhead = 2
learning_rate = 0.01

hidden_size = 256
input_size = dataset.eng_vocab.n_words
output_size = dataset.fr_vocab.n_words
encoder = Encoder(input_size, hidden_size, nhead, num_layers).to(device)
decoder = Decoder(hidden_size, output_size).to(device)

encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

In [ ]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=12):
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    # Initialize encoder_hidden for training
    encoder_hidden = encoder.initHidden()

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

    decoder_input = torch.tensor([[dataset.fr_vocab.word2index['<SOS>']]], device=device)
    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()

        loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
        if decoder_input.item() == dataset.fr_vocab.word2index['<EOS>']:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

def calculate_accuracy(predicted_indices, target_indices):
    return sum([1 for pred, target in zip(predicted_indices, target_indices) if pred == target]) / len(target_indices)

criterion = nn.CrossEntropyLoss()

n_epochs = 110

for epoch in range(n_epochs):
    total_loss = 0
    total_tokens = 0
    correct_tokens = 0

    encoder.train()
    decoder.train()

    for input_tensor, target_tensor, _, _ in dataloader:
        input_tensor = input_tensor[0].to(device)
        target_tensor = target_tensor[0].to(device)

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        total_loss += loss

        input_length = input_tensor.size(0)
        target_length = target_tensor.size(0)

        predicted_indices = []

        # Initialize encoder_hidden for training
        encoder_hidden = encoder.initHidden()

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

        decoder_input = torch.tensor([[dataset.fr_vocab.word2index['<SOS>']]], device=device)
        decoder_hidden = encoder_hidden

        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            predicted_indices.append(topi.item())
            decoder_input = topi.squeeze().detach()

            if decoder_input.item() == dataset.fr_vocab.word2index['<EOS>']:
                break

        correct_tokens += calculate_accuracy(predicted_indices, target_tensor.tolist())
        total_tokens += target_length

    val_total_loss = 0
    val_total_tokens = 0
    val_correct_tokens = 0

    encoder.eval()
    decoder.eval()

    with torch.no_grad():
        for input_tensor, target_tensor, _, _ in dataloader:
            input_tensor = input_tensor[0].to(device)
            target_tensor = target_tensor[0].to(device)

            # Initialize encoder_hidden for evaluation
            encoder_hidden = encoder.initHidden()

            input_length = input_tensor.size(0)
            target_length = target_tensor.size(0)

            loss = 0

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

            decoder_input = torch.tensor([[dataset.fr_vocab.word2index['<SOS>']]], device=device)
            decoder_hidden = encoder_hidden

            predicted_indices = []

            for di in range(target_length):
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                predicted_indices.append(topi.item())
                decoder_input = topi.squeeze().detach()

                loss += criterion(decoder_output, target_tensor[di].unsqueeze(0))
                if decoder_input.item() == dataset.fr_vocab.word2index['<EOS>']:
                    break

            val_total_loss += loss.item() / target_length
            val_correct_tokens += calculate_accuracy(predicted_indices, target_tensor.tolist())
            val_total_tokens += target_length

    if epoch % 10 == 0:
        print(f'Epoch {epoch}: Train Loss: {total_loss / len(dataloader):.4f}, Train Acc: {correct_tokens / total_tokens:.4f}, Val Loss: {val_total_loss / len(dataloader):.4f}, Val Acc: {val_correct_tokens / val_total_tokens:.4f}')

    if epoch == n_epochs - 1:
        print("\nPrediction Examples:")
        n_examples = 10
        with torch.no_grad():
            for i, (input_tensor, target_tensor, _, _) in enumerate(dataloader):
                input_tensor = input_tensor[0].to(device)
                target_tensor = target_tensor[0].to(device)

                encoder_hidden = encoder.initHidden()

                input_length = input_tensor.size(0)
                target_length = target_tensor.size(0)

                predicted_indices = []

                for ei in range(input_length):
                    encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

                decoder_input = torch.tensor([[dataset.fr_vocab.word2index['<SOS>']]], device=device)
                decoder_hidden = encoder_hidden

                for di in range(target_length):
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                    topv, topi = decoder_output.topk(1)
                    predicted_indices.append(topi.item())
                    decoder_input = topi.squeeze().detach()

                    if decoder_input.item() == dataset.fr_vocab.word2index['<EOS>']:
                        break

                if i < n_examples:
                    predicted_string = ' '.join([dataset.fr_vocab.index2word[index] for index in predicted_indices if index not in (dataset.fr_vocab.word2index['<SOS>'], dataset.fr_vocab.word2index['<EOS>'], dataset.fr_vocab.word2index['<PAD>'])])
                    target_string = ' '.join([dataset.fr_vocab.index2word[index.item()] for index in target_tensor if index.item() not in (dataset.fr_vocab.word2index['<SOS>'], dataset.fr_vocab.word2index['<EOS>'], dataset.fr_vocab.word2index['<PAD>'])])
                    input_string = ' '.join([dataset.eng_vocab.index2word[index.item()] for index in input_tensor if index.item() not in (dataset.eng_vocab.word2index['<SOS>'], dataset.eng_vocab.word2index['<EOS>'], dataset.eng_vocab.word2index['<PAD>'])])

                    print(f'Input: {input_string}, Target: {target_string}, Predicted: {predicted_string}')

# Calculate overall evaluation results
overall_val_loss = val_total_loss / len(dataloader)
overall_val_accuracy = val_correct_tokens / val_total_tokens
overall_train_loss = total_loss / len(dataloader)
overall_train_accuracy = correct_tokens / total_tokens

print("\nOverall Evaluation Results:")
print(f'Overall Training Loss: {overall_train_loss:.4f}, Overall Training Accuracy: {overall_train_accuracy:.4f}, Overall Validation Loss: {overall_val_loss:.4f}, Overall Validation Accuracy: {overall_val_accuracy:.4f}')

Epoch 0: Train Loss: 3.2101, Train Acc: 0.0098, Val Loss: 3.5996, Val Acc: 0.0341
Epoch 10: Train Loss: 3.3181, Train Acc: 0.0327, Val Loss: 3.2505, Val Acc: 0.0341
Epoch 20: Train Loss: 3.2888, Train Acc: 0.0321, Val Loss: 3.1530, Val Acc: 0.0341
Epoch 30: Train Loss: 3.2585, Train Acc: 0.0325, Val Loss: 3.5581, Val Acc: 0.0083
Epoch 40: Train Loss: 3.2571, Train Acc: 0.0311, Val Loss: 3.5171, Val Acc: 0.0083
Epoch 50: Train Loss: 3.2406, Train Acc: 0.0317, Val Loss: 3.1588, Val Acc: 0.0341
Epoch 60: Train Loss: 3.2355, Train Acc: 0.0324, Val Loss: 3.1673, Val Acc: 0.0341
Epoch 70: Train Loss: 3.2382, Train Acc: 0.0328, Val Loss: 3.5328, Val Acc: 0.0083
Epoch 80: Train Loss: 3.2300, Train Acc: 0.0322, Val Loss: 3.4664, Val Acc: 0.0083
Epoch 90: Train Loss: 3.2311, Train Acc: 0.0327, Val Loss: 3.1724, Val Acc: 0.0341
Epoch 100: Train Loss: 3.2352, Train Acc: 0.0312, Val Loss: 3.4219, Val Acc: 0.0083

Prediction Examples:
Input: We build a sandcastle, Target: Nous construisons un châtea

In [ ]:
# Define the French sentences for translation
french_sentences = [
    "J'ai froid",
    "Tu es fatigué",
    "Il a faim",
    "Elle est heureuse",
    "Nous sommes amis",
    "Ils sont étudiants",
    "Le chat dort",
    "Le soleil brille",
    "Le bébé pleure"

]

In [ ]:
# Define some French sentences for translation
print(" ")
print("English translations for some French sentences:")


# Tokenize and pad the French sentences
tokenized_french_sentences = tokenize_and_pad(french_sentences, dataset.fr_vocab)

# Convert tokenized sentences into tensors
input_tensors = tokenized_french_sentences.to(device)

# Generate English translations
with torch.no_grad():
    for input_tensor in input_tensors:
        # Initialize encoder hidden states
        encoder_hidden = encoder.initHidden()

        input_length = input_tensor.size(0)

        # Pass input through the encoder
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei].unsqueeze(0), encoder_hidden)

        # Initialize decoder input with SOS token
        decoder_input = torch.tensor([[dataset.eng_vocab.word2index['<SOS>']]], device=device)
        decoder_hidden = encoder_hidden

        # Initialize list to store predicted indices
        predicted_indices = []

        # Generate translation
        for _ in range(12):  # Assuming max length is 12
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            predicted_indices.append(topi.item())

            if predicted_indices[-1] == dataset.eng_vocab.word2index['<EOS>']:
                break

            decoder_input = topi.squeeze().detach()

        # Convert predicted indices to English words
        predicted_words = [dataset.eng_vocab.index2word[index] for index in predicted_indices if
                           index not in (dataset.eng_vocab.word2index['<SOS>'], dataset.eng_vocab.word2index['<EOS>'], dataset.eng_vocab.word2index['<PAD>'])]

        # Print the translations
        print("French Sentence:", ' '.join([dataset.fr_vocab.index2word[index.item()] for index in input_tensor if
                                              index.item() not in (dataset.fr_vocab.word2index['<SOS>'], dataset.fr_vocab.word2index['<EOS>'], dataset.fr_vocab.word2index['<PAD>'])]))

 
English translations for some French sentences:
French Sentence: J'ai froid
French Sentence: Tu es fatigué
French Sentence: Il a faim
French Sentence: Elle est heureuse
French Sentence: Nous sommes amis
French Sentence: Ils sont étudiants
French Sentence: Le chat dort
French Sentence: Le soleil brille
French Sentence: Le bébé pleure
